In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

#import matplotlib.pyplot as plt



torch.manual_seed(0)

In [2]:
import numpy as np
ids=get_ids_in_folder(base_path="./data/public/1.0.1/test/")

idx=10
data=load_npz_by_id(idx,base_path="./data/public/1.0.1/test/")



#data=np.load("path")
tmp_name=data["alos"][idx] #shape : 5000;4;4;4;3 (batch / time / h / w / c)
print(tmp_name.shape)

#3456
#11520
#1920
#420
#192



(4, 4, 4, 3)


In [3]:
#create_dataset(name="tiny", mode="validation",max_imgs=-1)

In [4]:
config=read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
transconfig=transformations_config_flair("./data/bands_info/bands.yaml",config)

dataset=CustomPlanted("./data/custom_planted/tiny_train.h5", config=config, trans_config=transconfig)

In [5]:
dataset[3]

(tensor([[1.5973e-06, 8.2588e-02, 9.9997e-01,  ..., 1.3691e-01, 9.9058e-01,
          4.3716e-02],
         [1.5973e-06, 8.2588e-02, 9.9997e-01,  ..., 1.3691e-01, 9.9058e-01,
          4.3716e-02],
         [1.5973e-06, 8.2588e-02, 9.9997e-01,  ..., 1.3691e-01, 9.9058e-01,
          4.3716e-02],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([1., 1., 1.,  ..., 0., 0., 0.]),
 np.int64(14),
 np.int64(0))

In [6]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torch
import os

seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
labels = load_json_to_dict("./data/labels.json")
bands_yaml = "./data/bands_info/bands.yaml"

trans_config = transformations_config_flair(bands_yaml, config_model)
xp_name = "test_Atos"

wand = False
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=get_xp_name(config_model['encoder']) + " modalities",
            project="Atos_modalities",
            config=config_model
        )
        wandb_logger = WandbLogger(project="tiny_modalities")

model = Model(config_model, wand=wand, name=xp_name, labels=labels)

data_module = CustomPlantedDataModule(
    "./data/custom_planted/tiny",
    config=config_model,
    trans_config=trans_config,
    batch_size=config_model['dataset']['batchsize'],
)

data_module.setup()


# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/",
    filename="best_model-{epoch:02d}-{val_IoU:.4f}",
    monitor="val_IoU",
    mode="max",
    save_top_k=1,
    verbose=True,
)

early_stop_callback = EarlyStopping(
    monitor="val_IoU",
    min_delta=0.00,
    patience=15,
    verbose=False,
    mode="max"
)



# Trainer
trainer = Trainer(
    use_distributed_sampler=False,
    #strategy="ddp",
    #devices=-1,
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=1,
    accelerator="gpu",
    callbacks=[early_stop_callback, checkpoint_callback],
    default_root_dir="./checkpoints/"
)


# Fit the model
trainer.fit(model, datamodule=data_module)


Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/hugoriffaud/Documents/PLANTED/checkpoints exists and is not empty.

   | Name                      | Type                      | Params | Mode 
-------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]Validation DataLoader created on rank: 0


/Users/hugoriffaud/Documents/PLANTED/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]torch.Size([1, 17508])


IndexError: too many indices for tensor of dimension 3